**pandapower – A Short Introduction**

This tutorial explains **panda**power library basis used in RHT laboratories. **panda**power is an easy to use tool for loadflow and short-circuit calculations in power systems. To go further, we recommend you take a look at these two
- [**panda**power's documentation](https://pandapower.readthedocs.io/en/v2.13.1/elements/trafo.html).
- [**panda**power's tutorials](https://www.pandapower.org/start/#interactive-tutorials-).
- [**panda**power_heig_ui's documentation](https://heig-vd-iese.github.io/pandapower-heig-ui/).

# Create a small power network

We consider the following simple 3-bus example network from [**panda**power's tutorial](https://github.com/e2nIEE/pandapower/blob/master/tutorials/minimal_example.ipynb).

<img alt="pandapower with 2-bus minimal example" width ="600" caption="Figure 1 – A minimal example with a 2-bus power system including PQ-load" src="https://github.com/e2nIEE/pandapower/raw/develop/tutorials/pics/3bus-system.png" id="pandapower_simple"/>

The above network can be created in pandapower as follows:

In [1]:
import pandapower as pp

## Create a **panda**power empty power network object

In order to create an empty network object, we can run the following **panda**power command:

In [2]:
# Create an empty network
net = pp.create_empty_network()
net

This pandapower network is empty

The empty network object is composed by a dictionary of pandas DataFrame.

## Create three buses with different voltage levels

We need three buses with different voltage levels and names, buses are the elements which connect equipments together:

In [3]:
# Create buses
bus1 = pp.create_bus(net, vn_kv=20., name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=0.4, name="Bus 2")
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")

**Remark**: We need to pay attention that voltage levels are express in kV and correspond to line voltages.

## Creating a transformer and connecting it to the network

For a full understanding of the parameters to be applied, please read the [documentation](https://pandapower.readthedocs.io/en/v2.13.1/elements/trafo.html) carefully. 

In order to create the transformer object that will be connection to the network previously created, we can proceed as follows:

In [4]:
# Create transformer
trafo = pp.create_transformer_from_parameters(
    net, hv_bus=bus1, lv_bus=bus2, sn_mva=0.4, vn_hv_kv=20.0, vn_lv_kv=0.4, vk_percent=6.0, vkr_percent=1.425, pfe_kw=1.35, i0_percent=0.3375, name="Trafo")

# trafo = pp.create_transformer(net, hv_bus=bus1, lv_bus=bus2, std_type="0.4 MVA 20/0.4 kV", name="Trafo")

**Remark**: pay attention in parameters units and in voltage levels matchs.

### Create a transmission line and connect it to the network

In [5]:
line = pp.create_line_from_parameters(net, from_bus=bus2, to_bus=bus3, length_km=0.1, r_ohm_per_km=0.642, x_ohm_per_km=0.083, c_nf_per_km=210, max_i_ka=0.142, name="Line")

### Create a load connect it to the network

In [6]:
# Create bus elements
load = pp.create_load(net, bus=bus3, p_mw=0.100, q_mvar=0.05, name="Load")

### Create an external grid connection

This element is mandatory to be able to perform powerflow simulations. It insures to keep powers balanced within the power network:

In [7]:
ext_grid = pp.create_ext_grid(net, bus=bus1, vm_pu=1.20, name="Grid Connection")

# Data structure and data access

A **panda**power network object is structured as a dictionary:

- Keys are the type names of power network equipments names such as line, load transformer, etc. (string).
- Values are tables which contains all the information needed about their corresponding equipments (pandas DataFrame).
 
By calling the network have a quick overview of it and the number of element for each equipment.

In [8]:
net

This pandapower network includes the following parameter tables:
   - bus (3 elements)
   - load (1 element)
   - ext_grid (1 element)
   - line (1 element)
   - trafo (1 element)

There are two ways to get the one equipment type table:

- By using the dictionary way to call values.
- By using the pandapower object.

In [9]:
net["bus"]

,name,vn_kv,type,zone,in_service
0,Bus 1,20.0,b,None,True
1,Bus 2,0.4,b,None,True
2,Bus 3,0.4,b,None,True


In [10]:
type(net["bus"])

pandas.core.frame.DataFrame

In [11]:
net.bus

,name,vn_kv,type,zone,in_service
0,Bus 1,20.0,b,None,True
1,Bus 2,0.4,b,None,True
2,Bus 3,0.4,b,None,True


In [12]:
net.trafo

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,i0_percent,shift_degree,tap_side,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service
0,Trafo,None,0,1,0.4,20.0,0.4,6.0,1.425,1.35,0.3375,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,False,1,1.0,True


In [13]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,Line,None,1,2,0.1,0.642,0.083,210.0,0.0,0.142,1.0,1,None,True


In [14]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,Load,2,0.1,0.05,0.0,0.0,NaN,1.0,True,wye


To have access to one specific element or value of a table, use Pandas functions:

In [15]:
net.bus.loc[0, :]

name          Bus 1
vn_kv          20.0
type              b
zone           None
in_service     True
Name: 0, dtype: object

In [16]:
type(net.bus.loc[0, :])

pandas.core.series.Series

In [17]:
net.bus.at[0, "name"]

'Bus 1'

We can also modify the data using Pandas function:

In [18]:
net.bus.loc[0, "name"] = "hv_bus"
net.bus

,name,vn_kv,type,zone,in_service
0,hv_bus,20.0,b,None,True
1,Bus 2,0.4,b,None,True
2,Bus 3,0.4,b,None,True


# Run power flow

Now we can run a balanced power flow calculation using the following command:

In [19]:
pp.runpp(net)
net

This pandapower network includes the following parameter tables:
   - bus (3 elements)
   - load (1 element)
   - ext_grid (1 element)
   - line (1 element)
   - trafo (1 element)
 and the following results tables:
   - res_bus (3 elements)
   - res_line (1 element)
   - res_trafo (1 element)
   - res_ext_grid (1 element)
   - res_load (1 element)

Then if you check you **panda**power object you will see that powerflow results tables have been added.

It may also be interesting to consult the results for buses, lines and transformers:

In [20]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.200000,0.000000,-0.106038,-0.051875
1,1.190635,-0.539841,0.000000,0.000000
2,1.153532,0.080701,0.100000,0.050000


In [21]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.103769,0.050486,-0.1,-0.05,0.003769,0.000486,0.139895,0.139896,0.139896,1.190635,-0.539841,1.153532,0.080701,98.518141


In [22]:
net.res_trafo

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,0.106038,0.051875,-0.103769,-0.050486,0.002268,0.001389,0.00284,0.139895,1.2,0.0,1.190635,-0.539841,24.593091


All other pandapower elements and power grid analysis functionality (e.g. optimal power flow, state estimation or short-circuit calculation) are also fully integrated into pandapower's tabular data structure. This concludes a short walkthrough of some pandapower features. More in-depth tutorials can be found under this [link](https://www.pandapower.org/start/#interactive-tutorials-.)

# Create small power network using pandapower_heig_ui package

A package has been created in order to simplify network generation, timeseries simulation and data visualisation. We can generate **panda**power object from data stored in excels files through the following function. We advice you to take a look at its [documentation](https://heig-vd-iese.github.io/pandapower-heig-ui/).

In [1]:
import pp_heig_plot as pp_plot
import pp_heig_simulation as pp_sim
from datetime import time

In [ ]:
net_file_path = "data/3_bus_example.xlsx"
net = pp_sim.load_net_from_xlsx(file_path=net_file_path)
net

We can plot a simplified diagram of our network using the following function:

- By adding a filename, the plot will be saved in a png format in the default folder _plot_.
- We can change the folder name using the folder parameter.
- We can view the equipment parameters in the plot by moving the mouse over them.
- The network is well traced when it is tree-like. In the case of a mesh grid, a coordinate parameter must be added to the buses.

In [ ]:
pp_plot.plot_power_network(net=net, plot_title="3-bus example", filename="3_bus_example")

We can run a simple power flow and visualise result using the following functions:

In [ ]:
pp.runpp(net)
pp_plot.plot_powerflow_result(net=net, plot_title="3-bus powerflow results", filename="3_bus_pp_result")
net.res_bus

## Timeseries powerflow simulation


We can create power profiles from excel files to perform timeseries powerflow simulations. After having been loaded, the resulting object is a dictionary of dataframe:

- Keys is the equipment name where profile are related to.
- Values can be active and reactive power profile table.

In [ ]:
profile_file_path = "data/3_bus_power_profile.xlsx"
time_series = pp_sim.load_power_profile_form_xlsx(file_path=profile_file_path)
print(time_series.keys())
print(time_series["load"].keys())
time_series["load"]["p_mw"]

In this example, the file loaded contains two different profiles for loads. If we take a look in the load **panda**power table we can see that the `profile_mapping` parameter of the load is set to 0. It means that power profiles applied to this load will be the 0.

In [ ]:
net.load

In [ ]:
pp_sim.apply_power_profile(net=net, equipment="load", power_profiles=time_series["load"])

Then we need to create an output writer which will store simulation results:

- Default results stored are `res_bus.vm_pu`, `res_line.loading_percent`, `res_trafo.loading_percent`.
- We can add other results using `add_results` parameters.

In [ ]:
pp_sim.create_output_writer(net=net, add_results= ["res_line.p_from_mw"])

Finally, we can run times series simulation and plot results – as follows:

In [ ]:
result_df = pp_sim.run_time_simulation(net=net)
print()
pp_plot.plot_timeseries_result(data_df=result_df["res_bus.vm_pu"], ylabel="V [pu]",
                       plot_title="Bus voltage", filename= "voltage_result")
print()
pp_plot.plot_timeseries_result(data_df=result_df["res_line.p_from_mw"], ylabel="P [MW]",
                       plot_title="line power", filename= "line_result")
print()
pp_plot.plot_timestamps_powerflow_result(net=net, filename="net_result_12h", plot_time=time(hour=12))

We can use the second power profile loaded for the excel file. To do this, we just need to modify the `profile_mapping` parameter before applying once again the power profile:

In [ ]:
net.load.loc[0, "profile_mapping"] = 1
pp_sim.apply_power_profile(net=net, equipment="load", power_profiles=time_series["load"])
result_df = pp_sim.run_time_simulation(net=net)
print()
pp_plot.plot_timeseries_result(data_df=result_df["res_bus.vm_pu"], ylabel="V [pu]",
                       plot_title="Bus voltage", filename= "voltage_result")
print()
pp_plot.plot_timeseries_result(data_df=result_df["res_line.p_from_mw"], ylabel="P [MW]",
                       plot_title="line power", filename= "line_result")

We can also scale our power profiles modifying `scaling` parameters:

In [ ]:
net.load.loc[0, "scaling"] = 5
result_df = pp_sim.run_time_simulation(net=net)
pp_plot.plot_timeseries_result(data_df=result_df["res_bus.vm_pu"], ylabel="V [pu]",
                       plot_title="Bus voltage", filename= "voltage_result")
print()
pp_plot.plot_timeseries_result(data_df=result_df["res_line.p_from_mw"], ylabel="P [MW]",
                       plot_title="line power", filename= "line_result")

In [ ]:
sc.calc_sc(net, fault=fault, ip=True)

# References

- [Pandapower 'Getting started'](http://www.pandapower.org/start/)
- [Pandapower's documentation](https://pandapower.readthedocs.io/en/v2.13.1/index.html)
- [Pandapower's tutorials on GitHub](https://github.com/e2nIEE/pandapower/tree/v2.13.1/tutorials)

## Citing pandapower

 &copy; Copyright 2016-2023 by Fraunhofer IEE and University of Kassel. Revision 2feba868.

```latex
@article{pandapower.2018,
author={L. Thurner and A. Scheidler and F. Schafer and J. H. Menke and J. Dollichon and F. Meier and S. Meinecke and M. Braun},
journal={IEEE Transactions on Power Systems},
title={pandapower - an Open Source Python Tool for Convenient Modeling, Analysis and Optimization of Electric Power Systems},
year={2018},
doi={10.1109/TPWRS.2018.2829021},
url={https://arxiv.org/abs/1709.06743},
ISSN={0885-8950}
}
```